# Create db

In [38]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup

In [39]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
#embeddings = HuggingFaceEmbeddings(model_name='dangvantuan/sentence-camembert-large', model_kwargs={'device': 'cpu'})

In [40]:
#from sentence_transformers import SentenceTransformer
#embeddings =  SentenceTransformer("dangvantuan/sentence-camembert-large")

In [41]:
def _preprocess_text(text: str) -> str:
        soup = BeautifulSoup(text, "html.parser").text
        soup_tab = soup.expandtabs()
        cleaned_text = ""

        for line in soup_tab.split("\n"):
            if line:
                cleaned_text += ". " + line

        return cleaned_text

In [42]:
def create_chunks(text, chunk_size=1000):
    # Create Document object from text documents 
    docs = [Document(page_content=post) for post in [text]]

    # Split documents into chunks 

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=10, separators=['\n\n', '\n', '.']
    )
    docs = text_splitter.split_documents(docs)
    return docs

## Store Embedings

In [43]:
def store_embeddings(docs):
    db = FAISS.from_documents(docs, embeddings)
    return db

# Récupération du modèle

In [72]:
from langchain.llms import CTransformers
llm = CTransformers(model="models/llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type='llama',
                    config={'max_new_tokens': 256,
                            'temperature': 0.01}
                   )

# Définition des fonctions

In [87]:
def build_qa_system(text, prompt):
    chunk_size=500
    vector_count=2
    #cleaned_text = _preprocess_text(text)
    docs = create_chunks(text, chunk_size)
    db = store_embeddings(docs)
    qa = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': vector_count}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                      )
    return qa

In [74]:
def _post_processing(qa_answer):
    if qa_answer.startswith("Réponse : "):
        return qa_answer.split("Réponse : ")[1]
    elif qa_answer.startswith("Votre réponse : "):
        return qa_answer.split("Votre réponse : ")[1]
    elif qa_answer.startswith("Answer : "):
        return qa_answer.split("Answer : ")[1]
    else:
        return qa_answer

In [75]:
def data_extraction(qa, query):
    res = qa({'query': query})
    answer = _post_processing(res["result"])
    print(answer)
    return answer

In [76]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

def explication(text, prompt_personna_template):
    prompt = PromptTemplate.from_template(prompt_personna_template)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    
    docs = [Document(page_content=post) for post in [text]]
    
    stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
    
    synthesis = stuff_chain.run(docs)
    
    return synthesis

In [82]:
from langchain.chains.summarize import load_summarize_chain

def summarize(text, template_resume):
    prompt = PromptTemplate(template=template_resume, input_variables=['text'])
    docs = [Document(page_content=post) for post in [text]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt, verbose=True)
    resume = chain.run(docs)
    return resume

In [78]:
from transformers import pipeline

analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

In [79]:
def is_positif(text):
    result = analyzer(text,
    return_all_scores=False)
    
    if any(v in result[0]['label'] for v in ['3','4','5']):
        return True
    else:
        return False

In [80]:
def extract_decision_component(input_string, start_line, end_line=None):
    start_index = input_string.find(start_line)
    if start_index == -1:
        return "Start line not found in the input string."

    start_index += len(start_line)

    if end_line is not None:
        end_index = input_string.find(end_line, start_index)
        if end_index == -1:
            return "End line not found in the input string after the start line."
    else:
        end_index = len(input_string)

    return input_string[start_index:end_index].strip()

# Jeu de données

In [53]:
text_decision = """
RÉPUBLIQUE FRANCAISE
AU NOM DU PEUPLE FRANCAIS
Vu la procédure suivante :

Procédure contentieuse antérieure :

M. C... a demandé au tribunal administratif de Toulouse de l'admettre au bénéfice de l'aide juridictionnelle provisoire, d'annuler l'arrêté du 17 février 2021 par lequel le préfet de la Haute-Garonne a refusé de lui accorder un titre de séjour, l'a obligé à quitter le territoire français dans un délai de trente jours et a fixé le pays de renvoi, d'enjoindre au préfet de la Haute-Garonne de lui délivrer le titre de séjour sollicité dans un délai d'un mois à compter de la notification du jugement à intervenir, sous astreinte de 100 euros par jours de retard ou, à tout le moins, de procéder au réexamen de sa demande, et de mettre à la charge de l'Etat la somme de 2 000 euros à verser à son conseil sur le fondement des dispositions des articles L. 761-1 du code de justice administrative et 37 alinéa 2 de la loi du 10 juillet 1991 relative à l'aide juridique, sous réserve que ce dernier renonce au bénéfice de l'aide juridictionnelle. 

Par un jugement n°2101626 du 1er avril 2022, le tribunal administratif de Toulouse a prononcé un non-lieu à statuer sur les conclusions de M. C... tendant à son admission au bénéfice de l'aide juridictionnelle et a rejeté le surplus de ses demandes.

Procédure devant la cour :

Par une requête, enregistrée le 1er décembre 2022, et des pièces complémentaires enregistrées le 24 mars 2023, M. C..., représenté par Me Brel, demande à la cour :

1°) d'annuler ce jugement du 1er avril 2022 ;

2°) d'annuler l'arrêté du préfet de la Haute-Garonne du 17 février 2021 ;

3°) d'enjoindre au préfet de la Haute-Garonne de lui délivrer le titre de séjour sollicité dans un délai d'un mois à compter de la notification du jugement à intervenir, sous astreinte de 100 euros par jours de retard ou, à tout le moins, de procéder au réexamen de sa demande ;

4°) de mettre à la charge de l'Etat les entiers dépens et la somme de 2 000 euros à verser à son conseil sur le fondement des dispositions des articles L. 761-1 du code de justice administrative et 37 alinéa 2 de la loi du 10 juillet 1991 relative à l'aide juridique. 

Il soutient que :

Sur la décision portant refus de titre de séjour : 
-la décision est entachée d'une erreur de droit et d'une erreur manifeste d'appréciation au regard des dispositions de l'article L. 313-11 11° du code de l'entrée et du séjour des étrangers et du droit d'asile ;
-la décision est entachée d'une erreur manifeste d'appréciation de sa situation et des conséquences qu'elle emporte sur sa situation personnelle ;
Sur la décision portant obligation de quitter le territoire français :
-la décision portant obligation de quitter le territoire français est privée de base légale en raison de l'illégalité de la décision portant refus de titre de séjour ;
-la décision est entachée d'une erreur de droit en ce qu'elle méconnaît les dispositions de l'article L. 511-4 10° du code de l'entrée et du séjour des étrangers et du droit d'asile ;
Sur la décision fixant le pays de renvoi :
-la décision fixant le pays de renvoi est dépourvue de base légale en raison de l'illégalité de la mesure d'éloignement prise à son encontre ;
-eu égard aux risques auxquels il est exposé en cas de retour dans son pays d'origine, la décision méconnaît l'article 3 de la convention européenne de sauvegarde des droits de l'homme et des libertés fondamentales et l'article L. 513-2 du code de l'entrée et du séjour des étrangers et du droit d'asile.

Par un mémoire en défense enregistré le 24 février 2023, le préfet de la Haute-Garonne conclut au rejet de la requête.

Il soutient que les moyens soulevés par l'appelant ne sont pas fondés.
Par ordonnance du 24 mars 2023, la clôture d'instruction a été fixée au 17 avril 2023. 

M. C... a été admis au bénéfice de l'aide juridictionnelle totale par décision du bureau d'aide juridictionnelle près le tribunal judiciaire de Toulouse du 9 novembre 2022. 
Vu les autres pièces du dossier.

Vu :
- la convention européenne de sauvegarde des droits de l'homme et des libertés fondamentales ;
- le code de l'entrée et du séjour des étrangers et du droit d'asile ;
- la loi n° 91-647 du 10 juillet 1991 modifiée relative à l'aide juridique ;
- l'arrêté du 5 janvier 2017 fixant les orientations générales pour l'exercice par les médecins de l'Office français de l'immigration et de l'intégration, de leurs missions, prévues à l'article L. 313-11 (11°) du code de l'entrée et du séjour des étrangers et du droit d'asile ;
- le code de justice administrative.

Les parties ont été régulièrement averties du jour de l'audience.

La présidente de la formation de jugement a dispensé la rapporteure publique, sur sa proposition, de prononcer des conclusions à l'audience.

Le rapport de Mme Armelle Geslan-Demaret, présidente rapporteure, a été entendu au cours de l'audience publique.


Considérant ce qui suit :

1. M. B... C..., ressortissant nigérian est entré sur le territoire français le 4 octobre 2018. Après annulation par le tribunal administratif de Toulouse, par un jugement du 24 janvier 2019, de l'arrêté du 21 janvier 2019 par lequel le préfet de la Haute-Garonne a décidé de son transfert aux autorités italiennes responsables de l'examen de sa demande d'asile, M. C... a sollicité l'asile le 25 octobre 2018 et sa demande d'asile a été rejetée de façon définitive par la Cour nationale du droit d'asile le 2 septembre 2020. Par un arrêté du 9 octobre 2020 le préfet de la Haute-Garonne l'a obligé à quitter le territoire français dans un délai de trente jours et a fixé le pays dont il a la nationalité en tant que pays de renvoi. Par un jugement du 31 décembre 2020, le tribunal administratif de Toulouse a annulé cet arrêté, au motif que cette décision d'éloignement était intervenue avant que le préfet n'ait statué sur la demande de titre de séjour en qualité d'étranger malade présentée par l'intéressé. M. C... a présenté le 13 octobre 2020 une demande de titre de séjour en qualité d'étranger malade sur le fondement des dispositions du 11° de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile. Par un arrêté du 17 février 2021, le préfet de la Haute-Garonne a refusé de lui délivrer un titre de séjour en qualité d'étranger malade, l'a obligé à quitter le territoire français dans un délai de trente jours et a fixé le pays de destination de la mesure d'éloignement. M. C... relève appel du jugement du 1er avril 2022 du tribunal administratif de Toulouse qui a rejeté sa demande tendant à l'annulation de cet arrêté.
Sur le bien-fondé du jugement :

2. Aux termes de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile, alors en vigueur : " Sauf si sa présence constitue une menace pour l'ordre public, la carte de séjour temporaire portant la mention " vie privée et familiale " est délivrée de plein droit : (...) 11° À l'étranger résidant habituellement en France, si son état de santé nécessite une prise en charge médicale dont le défaut pourrait avoir pour lui des conséquences d'une exceptionnelle gravité et si, eu égard à l'offre de soins et aux caractéristiques du système de santé dans le pays dont il est originaire, il ne pourrait pas y bénéficier effectivement d'un traitement approprié. La condition prévue à l'article L. 313-2 n'est pas exigée. La décision de délivrer la carte de séjour est prise par l'autorité administrative après avis d'un collège de médecins du service médical de l'Office français de l'immigration et de l'intégration, dans des conditions définies par décret en Conseil d'État. Les médecins de l'office accomplissent cette mission dans le respect des orientations générales fixées par le ministre chargé de la santé. (...) ". Aux termes de l'article 4 de l'arrêté du 5 janvier 2017 fixant les orientations générales pour l'exercice par les médecins de l'Office français de l'immigration et de l'intégration de cette mission : " Les conséquences d'une exceptionnelle gravité résultant d'un défaut de prise en charge médicale, mentionnées au 11° de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile, sont appréciées sur la base des trois critères suivants : degré de gravité (mise en cause du pronostic vital de l'intéressé ou détérioration d'une de ses fonctions importantes), probabilité et délai présumé de survenance de ces conséquences. Cette condition des conséquences d'une exceptionnelle gravité résultant d'un défaut de prise en charge doit être regardée comme remplie chaque fois que l'état de santé de l'étranger concerné présente, en l'absence de la prise en charge médicale que son état de santé requiert, une probabilité élevée à un horizon temporel qui ne saurait être trop éloigné de mise en jeu du pronostic vital, d'une atteinte à son intégrité physique ou d'une altération significative d'une fonction importante. Lorsque les conséquences d'une exceptionnelle gravité ne sont susceptibles de ne survenir qu'à moyen terme avec une probabilité élevée (pathologies chroniques évolutives), l'exceptionnelle gravité est appréciée en examinant les conséquences sur l'état de santé de l'intéressé de l'interruption du traitement dont il bénéficie actuellement en France (rupture de la continuité des soins). Cette appréciation est effectuée en tenant compte des soins dont la personne peut bénéficier dans son pays d'origine ".

3. Sous réserve des cas où la loi attribue la charge de la preuve à l'une des parties, il appartient au juge administratif saisi de l'affaire, au vu des pièces du dossier et compte tenu, le cas échéant, de l'abstention d'une des parties à produire les éléments qu'elle est seule en mesure d'apporter et qui ne sauraient être réclamés qu'à elle-même, d'apprécier si l'état de santé d'un étranger nécessite une prise en charge médicale dont le défaut pourrait entraîner pour lui des conséquences d'une exceptionnelle gravité, sous réserve de l'absence d'un traitement approprié dans le pays de renvoi et de la possibilité d'y bénéficier effectivement d'un traitement approprié, sauf circonstance humanitaire exceptionnelle. La partie qui justifie d'un avis du collège de médecins du service médical de l'Office français de l'immigration et de l'intégration qui lui est favorable doit être regardée comme apportant des éléments de fait susceptibles de faire présumer l'existence ou l'absence d'un état de santé de nature à justifier la délivrance ou le refus d'un titre de séjour. Dans ce cas, il appartient à l'autre partie, dans le respect des règles relatives au secret médical, de produire tous éléments permettant d'apprécier l'état de santé de l'étranger et, le cas échéant, l'existence ou l'absence d'un traitement approprié dans le pays de renvoi et la possibilité d'en bénéficier effectivement. La conviction du juge, à qui il revient d'apprécier si l'état de santé d'un étranger justifie la délivrance d'un titre de séjour dans les conditions ci-dessus rappelées, se détermine au vu de ces échanges contradictoires.

4. Pour refuser à M. C... la délivrance d'un titre de séjour sur le fondement des dispositions du 11° de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile, le préfet de la Haute-Garonne s'est notamment fondé sur l'avis du 23 décembre 2020 par lequel le collège de médecins de l'Office français de l'immigration et de l'intégration a estimé que, si l'état de santé de l'intéressé nécessitait une prise en charge médicale, le défaut d'une telle prise en charge ne devrait pas entraîner des conséquences d'une exceptionnelle gravité, et qu'il pouvait voyager sans risque vers son pays d'origine.

5. Il ressort des pièces du dossier que M. C... souffre d'un état de stress post-traumatique à la suite des événements vécus au Nigéria et présente un syndrome dépressif majeur avec un risque suicidaire moyen à élevé, pour lesquels il fait l'objet d'un suivi médical et d'un traitement médicamenteux. Si les certificats médicaux du 10 mars 2021 et du 28 juin 2022, qui font état d'une détérioration significative de l'état de santé du requérant, sont postérieurs à l'arrêté attaqué, il ressort toutefois du certificat médical du 15 septembre 2020, établi par le même praticien, le docteur A..., médecin attaché des hôpitaux de Toulouse, que l'état de santé de M. C... " nécessite une prise en charge dont le défaut pourrait avoir pour lui des conséquences graves, notamment le suicide " et que " dans l'éventualité où il devrait se rendre de nouveau au Nigéria, il pourrait avoir une recrudescence de la symptomatologie post-traumatique et un suicide dans le cadre d'un raptus anxieux est à craindre ". Dans ces conditions, les éléments produits par M. C..., dans les circonstances de l'espèce, sont de nature à remettre en cause l'appréciation portée par le collège des médecins de l'Office français de l'immigration et de l'intégration sur la gravité de son état de santé, et ainsi que le bien-fondé de l'appréciation du préfet qui se l'est approprié. Le requérant est, par suite, fondé à soutenir qu'en refusant de lui délivrer un titre de séjour en qualité d'étranger malade pour le motif évoqué au point 4, le préfet de la Haute-Garonne a méconnu le 11° de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile. 

6. Il résulte de ce qui précède, sans qu'il soit besoin de se prononcer sur les autres moyens de la requête, que M. C... est fondé à soutenir que c'est à tort que, par le jugement attaqué, le tribunal administratif de Toulouse a rejeté sa demande. 

Sur les conclusions à fin d'injonction sous astreinte : 

7. L'exécution du présent arrêt n'implique pas nécessairement la délivrance d'un titre de séjour à M. C..., mais seulement d'enjoindre au préfet de la Haute-Garonne de réexaminer sa situation dans le délai de deux mois à compter de la notification du présent arrêt, sans qu'il soit besoin d'assortir cette injonction d'une astreinte.

Sur les frais liés à l'instance :

8. M. C... ne justifie pas avoir engagé, dans la présente instance, des frais mentionnés à l'article R. 761-1 du code de justice administrative. Dès lors, ses conclusions tendant à la condamnation de l'Etat aux dépens ne peuvent qu'être rejetées.

9. Sous réserve de la renonciation de Me Brel à percevoir la somme correspondant à la part contributive de l'Etat, il y a lieu, dans les circonstances de l'espèce, de mettre à la charge de l'Etat sur le fondement de l'article L. 761-1 du code de justice administrative et de l'article 37 alinéa 2 de la loi du 10 juillet 1991 relative à l'aide juridique, le versement d'une somme de 1 000 euros au conseil de M. C... au titre des frais exposés et non compris dans les dépens. 

D E C I D E :

Article 1er : Le jugement du tribunal administratif de Toulouse du 1er avril 2022 est annulé.
Article 2 : L'arrêté du préfet de la Haute-Garonne du 17 février 2021 est annulé.
Article 3 : Il est enjoint au préfet de la Haute-Garonne de réexaminer la situation de M. C... dans le délai de deux mois à compter de la notification du présent arrêt.
Article 4 : L'Etat versera à Me Brel une somme de 1 000 euros en application de l'article 37 alinéa 2 de la loi du 10 juillet 1991 sous réserve de la renonciation de ce dernier à percevoir la somme correspondant à la part contributive de l'Etat.
Article 5 : Le surplus des conclusions de la requête est rejeté.
Article 6 : Le présent arrêt sera notifié à M. B... C..., à Me Brel et au ministre de l'intérieur et des outre-mer.
Copie en sera adressée pour information au préfet de la Haute-Garonne.
Délibéré après l'audience du 24 octobre 2023, à laquelle siégeaient :
Mme Geslan-Demaret, présidente de chambre,
Mme Blin, présidente assesseure,
M. Teulière, premier conseiller. 
Rendu public par mise à disposition au greffe le 14 novembre 2023.
La présidente rapporteure,
A. Geslan-Demaret La présidente assesseure,
A. Blin
"""

In [54]:
start_line = "RÉPUBLIQUE FRANCAISE"
end_line = "Considérant ce qui suit :"
text_litige = extract_decision_component(text_decision, start_line, end_line)

In [55]:
start_line = "Considérant ce qui suit :"
end_line = "D E C I D E :"
text_motif = extract_decision_component(text_decision, start_line, end_line)

In [56]:
start_line = "D E C I D E :"
end_line = None
text_decide = extract_decision_component(text_decision, start_line, end_line)

# Prompt

In [71]:
from langchain.prompts import PromptTemplate

qa_template = """
Contexte : {context}

Répondez à la question suivante en se basant sur le contexte : {question}
Donner la réponse directement et la traduire en français si besoin
"""

prompt = PromptTemplate(template=qa_template,
                            input_variables=['context', 'question'])

In [62]:
#___prompt explication______

template_explication = """
Expliquer dans un langage simple le texte suivant

Contexte : {text}

L'explication doit contenit au maximum 5 lignes
"""

In [63]:
#___prompt Titre______

template_titre = """Donner un titre au texte suivant :
Contexte : {text}
UNE SEULE LIGNE
"""

In [64]:
#___prompt Resume______

template_resume = """
Veuillez fournir un résumé succinct et précis de la décision judiciaire suivante

Contexte : {text}

Le resumé doit contenir au maximum 5 lignes
"""

prompt_resume = PromptTemplate(template=template_resume, input_variables=['context'])

In [65]:
#___prompt Patrice______
# Le texte suivant est extrait d'une décision de justice. 

template_patrice = """Expliquez dans un paragraphe concis le texte suivant à Patrice agé 60 ans, dirigeant d'une PME sans expertise juridique ni expérience contentieuse:
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

#___prompt Anna______

template_anna = """Expliquez dans un paragraphe concis le texte suivant à Hugo, âgé de 20 ans, qui souffre de déficiences cognitives. Son handicap ne lui permet que de lire et comprendre le français facile à lire et à comprendre FALC
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

#___prompt Hugo______
template_hugo = """Expliquez dans un paragraphe concis le texte suivant à Anna, âgée de 30 ans, qui est réfugiée et vit en France depuis 2022. Elle apprend le français et à un niveau de langue A2.
"{text}"
L'explication doit s'adresser directement à Patrice
UNE EXPLICATION CONCISE:"""

In [66]:
data = {
    "decision_complete": text_decision,
    "partie_litige": text_litige,
    "partie_motif": text_motif,
    "partie_resultat": text_decide
    
}

# Initialisation de la synthèse

In [67]:
synthesis = {
    "litige":
    {
        "quel est le requérant ?": "",
        "Qui a produit un mémoire en défense?": "",
        "De quoi le requérant se plaint-il (sans donner les arguments)?":"",
        "Le requérant veut-il que le juge annule une décision ?": "",
        "Le requérant veut-il que le juge lui octroie une somme d’argent ?": ""
    },
    "result":
    {
        "le requérant a-t-il  eu gain de cause ?":"",
        "Des sommes ont-elles été versées? Quel montant?":""
        
    },
    "motif":
    {
        "Pourquoi le requérant a-t-il échoué/réussi/non-lieu à statuer au regard du contexte (attention aux bonnes juridictions)?":""  
    }
    
}

In [88]:
synthesis = {
    "dispute":
    {
        "who is the plaintiff?": "",
        "who filed a defense memorandum?": "",
        "what does the plaintiff complain about (without providing the arguments)?": "",
        "does the plaintiff want the judge to overturn a decision?": "",
        "does the plaintiff want the judge to award them a sum of money?": ""
    },
    "result":
    {
        "did the plaintiff prevail?": "",
        "have any amounts been paid? If so, how much?": ""
        
    },
    "reason":
    {
        "why did the plaintiff fail/succeed/non-suit in the context (pay attention to the correct jurisdictions)?": ""  
    }
    
}

# Création de la synthèse

In [89]:
text_litige = """
M. C... requested the Administrative Court of Toulouse to grant him provisional legal aid, to annul the decree of February 17, 2021, by which the prefect of Haute-Garonne refused to grant him a residence permit, ordered him to leave French territory within thirty days, and specified the country of return. He also sought an injunction for the prefect of Haute-Garonne to issue the requested residence permit within one month from the notification of the forthcoming judgment, under a penalty of 100 euros per day of delay, or at least to re-examine his application. Additionally, M. C... sought to impose a fine of 2,000 euros on the State to be paid to his counsel, based on the provisions of articles L. 761-1 of the Administrative Justice Code and article 37 paragraph 2 of the law of July 10, 1991, concerning legal aid, provided that the latter waives the right to legal aid.

In judgment No. 2101626 dated April 1, 2022, the Administrative Court of Toulouse issued a decision of non-liability regarding M. C...'s request for legal aid and rejected the rest of his claims.

Proceedings before the court:

In a petition filed on December 1, 2022, and additional documents recorded on March 24, 2023, M. C..., represented by Mr. Brel, asks the court to:

Annul the judgment of April 1, 2022.
Annul the decree of the prefect of Haute-Garonne dated February 17, 2021.
Enjoin the prefect of Haute-Garonne to issue the requested residence permit within one month from the notification of the forthcoming judgment, under a penalty of 100 euros per day of delay, or at least to re-examine his application.
Impose on the State all costs and a fine of 2,000 euros to be paid to his counsel based on the provisions of articles L. 761-1 of the Administrative Justice Code and article 37 paragraph 2 of the law of July 10, 1991, concerning legal aid.
He argues that:

Regarding the decision refusing the residence permit:

The decision is tainted with a legal error and a manifest error of judgment in light of the provisions of Article L. 313-11 11° of the Code of Entry and Stay of Foreigners and Asylum Law.
The decision is tainted with a manifest error of judgment regarding his situation and the consequences it has on his personal situation.
Regarding the decision obliging him to leave French territory:

The decision obliging him to leave French territory lacks a legal basis due to the illegality of the decision refusing the residence permit.
The decision is tainted with a legal error as it disregards the provisions of Article L. 511-4 10° of the Code of Entry and Stay of Foreigners and Asylum Law.
Regarding the decision specifying the country of return:

The decision specifying the country of return lacks a legal basis due to the illegality of the deportation measure against him.
Given the risks he faces upon returning to his country of origin, the decision violates Article 3 of the European Convention on Human Rights and Fundamental Freedoms and Article L. 513-2 of the Code of Entry and Stay of Foreigners and Asylum Law.
In a defense memorandum filed on February 24, 2023, the prefect of Haute-Garonne requests the dismissal of the petition, arguing that the appellant's grounds are unfounded. By order of March 24, 2023, the close of the proceedings was set for April 17, 2023.

M. C... was granted total legal aid by the decision of the Legal Aid Office near the Toulouse Judicial Tribunal on November 9, 2022. Considering the other documents in the file.
"""

In [92]:
# Traitement de la partie litige
chunk_size = 500
vector_count = 2
#chunk_size = 400
#vector_count = 3

qa_template = """<s>[INST] <<SYS>> you are question answering system <</SYS>>
Context : {context}

Answer the following question based on the context: {question}[\INST]
"""

prompt = PromptTemplate(template=qa_template,
                            input_variables=['context', 'question'])

qa = build_qa_system(text_litige, prompt)

import time
start_time = time.time()

for query in synthesis["dispute"]:
    synthesis["dispute"][query] = data_extraction(qa, query)
    
print("--- %s seconds ---" % (time.time() - start_time))

The plaintiff in this case is M. C....
The prefect of Haute-Garonne filed a defense memorandum.
The plaintiff complains about the rejection of his request for legal aid and the dismissal of his claims.
Yes, the plaintiff wants the judge to overturn a decision. In the petition filed on December 1, 2022, and additional documents recorded on March 24, 2023, M. C... asks the court to "overturn" the decision of non-liability issued by the Administrative Court of Toulouse in Judgment No. 2101626 dated April 1, 2022.
Yes, the plaintiff wants the judge to award them a sum of money. According to the context, the plaintiff is seeking a fine of 2,000 euros to be paid to their counsel based on the provisions of articles L. 761-1 of the Administrative Justice Code and article 37 paragraph 2 of the law of July 10, 1991, concerning legal aid. This indicates that the plaintiff is asking for monetary compensation as a result of the State's failure to issue the requested residence permit within the requ

In [85]:
#___prompt explication______

text = """
Non, the requirant does not want the judge to award him any money. According to the context, the requirant is asking the court to annul a previous judgment and an administrative decision, and to order the prefect of the Haute-Garonne to issue a residence permit within a month's time. The requirant does not seek any monetary compensation or award.
"""

template_explication = """
Traduire le texte suivant en français, sans aucun ajout : {text}

"""

summarize(text, template_explication)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Traduire le texte suivant en français, sans aucun ajout : 
Non, the requirant does not want the judge to award him any money. According to the context, the requirant is asking the court to annul a previous judgment and an administrative decision, and to order the prefect of the Haute-Garonne to issue a residence permit within a month's time. The requirant does not seek any monetary compensation or award.




> Finished chain.

> Finished chain.


"Traduction en français :\nNon, le requérant ne souhaite pas que le juge lui attribue de l'argent. Selon le contexte, le requérant demande au tribunal d'annuler une décision judiciaire et administrative précédente et d'ordonner au préfet de la Haute-Garonne de délivrer un permis de résidence dans les un mois. Le requérant ne sollicite pas de compensation monétaire ou d'award."

In [70]:
# Traitement de la partie du résultat

# Création du système QA
chunk_size = 500
vector_count = 2
#chunk_size = 400
#vector_count = 3
qa = build_qa_system(text_decide, chunk_size, vector_count)

import time
start_time = time.time()

for query in synthesis["result"]:
    synthesis["result"][query] = data_extraction(qa, query)
    
print("--- %s seconds ---" % (time.time() - start_time))

Oui
Oui, des sommes ont été versées. Le montant est de 1 000 euros.
--- 30.706081867218018 seconds ---


In [18]:
# Traitement de la partie du motif

chunk_size = 500
vector_count = 2
#chunk_size = 400
#vector_count = 3
qa = build_qa_system(text_motif, chunk_size, vector_count)

#synthesis["motif"] = {}
#result = is_positif(synthesis["result"]["le requérant a-t-il  eu gain de cause ?"])
result = True
                                        
import time
start_time = time.time()

if result:
    query = "Peux-tu simplifier ?"
    #synthesis["motif"][query] = data_extraction(qa, query) 
    data_extraction(qa, query) 
else:
    query = "Pourquoi le requérant a-t-il subi une défaite ?"
    #synthesis["motif"][query] = data_extraction(qa, query)
    data_extraction(qa, query)
                                        
print("--- %s seconds ---" % (time.time() - start_time))

Le requérant est fondé à soutenir que le préfet de la Haute-Garonne a méconnu le 11° de l'article L. 313-11 du code de l'entrée et du séjour des étrangers et du droit d'asile en refusant de lui délivrer un titre de séjour en qualité d'étranger malade pour le motif évoqué au point 4.
--- 86.87802577018738 seconds ---


In [23]:
template_resume = """Expliquer le texte suivant:
"{text}"
"""

In [33]:
explication(text_motif, template_resume)

Number of tokens (555) exceeded maximum context length (512).
Number of tokens (556) exceeded maximum context length (512).
Number of tokens (557) exceeded maximum context length (512).
Number of tokens (558) exceeded maximum context length (512).
Number of tokens (559) exceeded maximum context length (512).
Number of tokens (560) exceeded maximum context length (512).
Number of tokens (561) exceeded maximum context length (512).
Number of tokens (562) exceeded maximum context length (512).
Number of tokens (563) exceeded maximum context length (512).
Number of tokens (564) exceeded maximum context length (512).
Number of tokens (565) exceeded maximum context length (512).
Number of tokens (566) exceeded maximum context length (512).
Number of tokens (567) exceeded maximum context length (512).
Number of tokens (568) exceeded maximum context length (512).
Number of tokens (569) exceeded maximum context length (512).
Number of tokens (570) exceeded maximum context length (512).
Number o

Number of tokens (688) exceeded maximum context length (512).
Number of tokens (689) exceeded maximum context length (512).
Number of tokens (690) exceeded maximum context length (512).
Number of tokens (691) exceeded maximum context length (512).
Number of tokens (692) exceeded maximum context length (512).
Number of tokens (693) exceeded maximum context length (512).
Number of tokens (694) exceeded maximum context length (512).
Number of tokens (695) exceeded maximum context length (512).
Number of tokens (696) exceeded maximum context length (512).
Number of tokens (697) exceeded maximum context length (512).
Number of tokens (698) exceeded maximum context length (512).
Number of tokens (699) exceeded maximum context length (512).
Number of tokens (700) exceeded maximum context length (512).
Number of tokens (701) exceeded maximum context length (512).
Number of tokens (702) exceeded maximum context length (512).
Number of tokens (703) exceeded maximum context length (512).
Number o

"The text is saying that Mr Celaire de l'\nThis text is a commet the text is a manque le textection de l'\nThe text is a violévaluable to challenge the text is a commettantici,\nThe text is a commet the d'\n\nThe text is a commet the text is saying that Mr.\nThis text is a commet the text is a manque le textection de l'\nThe text is a commet the text is a violévaluable to challenge the text is a commettantici,\nThe text is a commet the d'\nThe text is a commet the text is saying that Mr.\nThis text is a commet the text appears to challenge the text is a manque le textection de l'\n\n\n\n\n\n\n\n\nThe text is a violévaluable to challenge the text is a commettantici,\nThe text is a commet the d'\n\n\nThis text is saying that Mr.\nThe text is a commet the text is a manque le textection de l'\n\n\nThe text is a commet the text appears to challenge the text"